In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Read data files given in csv format.
train_data = pd.read_csv('../input/Kannada-MNIST/train.csv')#train_data.shape = (60000,785)
test_data = pd.read_csv('../input/Kannada-MNIST/test.csv') #test_data.shape = (5000,785)
dig_data = pd.read_csv('../input/Kannada-MNIST/Dig-MNIST.csv') #dig_data.shape = (10240,785)
sample_submission_data = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv') #sample_submission_data.shape = (5000,2)

In [ ]:
#Set Y values.
Y_train = train_data['label'] #Y_train.shape = (60000,)
Y_test = test_data['id'] #Y_test.shape = (5000,)
Y_dig = dig_data['label'] #Y_dig.shape = (10240,)

#Set X values.
X_train = train_data.drop(['label'], axis = 1) #X_train.shape = (60000,784)
X_test = test_data.drop(['id'], axis = 1) #X_test.shape = (5000,784)
X_dig = dig_data.drop(['label'], axis = 1) #X_dig.shape = (10240,785)


In [ ]:
#Show images in grayscale. For this aim we need to reshape images from 784 to 28x28.
fig, axes = plt.subplots(2, 5, figsize=(15,5))
for i,ax in enumerate(axes.flat):
    ax.imshow(X_train.to_numpy().reshape(60000, 28, 28)[i], cmap="gray") #colormap = grayscale
    

In [ ]:
#Convert from pandas dataframe to numpy array then reshape all data to (28x28x1) 3D matrices.
X_train = X_train.to_numpy().reshape(60000, 28, 28, 1) 
X_test = X_test.to_numpy().reshape(5000, 28, 28, 1)
X_dig = X_dig.to_numpy().reshape(10240, 28, 28, 1)

#Make sure that the values are float so that we can get decimal points after division.
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_dig = X_dig.astype('float32')

#Normalize input values by dividing with 255. Output values are in between 0 and 1.
#Rescale pixel values preferred for neural network models.
X_train /= 255.0
X_test /= 255.0

#Label encoding.
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes = 10, dtype='uint8')

# Create validation data.
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.15, random_state=42) #Test data size is 15% of train data size.

In [ ]:
#Creating a sequential model and adding the layers.
#CNN architecture consists of 6 layers. (Conv2D --> MaxPooling2D --> Flatten --> Dense --> Dropout --> Dense)
model = models.Sequential()
model.add(layers.Conv2D(28, kernel_size=(3,3), input_shape=(28,28,1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Flatten()) # Flattening the arrays for fully connected layers.
model.add(layers.Dense(128, activation=tf.nn.relu))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10,activation=tf.nn.softmax))

In [ ]:
model.compile(optimizer='adam', #Optimizer that implements the Adam algorithm which is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.
              loss='categorical_crossentropy', #The categorical_crossentropy computes the cross-entropy loss between the true classes and predicted classes.
              metrics=['accuracy']) #Metrics is a function that is used to judge the performance of your model.Accuracy metrics calculates how often predictions equal labels.
model.fit(x=X_val,y=Y_val, epochs=10)

In [ ]:
#Check architecture of the model.
model.summary()

In [ ]:
#Evaluate the model.
model.evaluate(X_val, Y_val) #Achieved a test accuracy of over 99%.

In [ ]:
#Model will classify the image as a ‘4’ and here is the visual of the image.
image_index = 3456 #Change image_index and show other numbers.
plt.imshow(X_val[image_index].reshape(28, 28),cmap='gray')
pred = model.predict(X_val[image_index].reshape(1, 28, 28, 1))
print(pred.argmax())